# **HOUSE PRICE PREDICTION**

Sử dụng các mô hình đã học hoặc nâng cao để áp dụng vào bài toán dự đoán giá nhà

In [51]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [52]:
train_df = pd.read_csv('../preprocessing_data/train.csv')
train_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.424462,-0.087206,-0.133270,0.651479,-0.517200,1.050994,0.878668,1.193074,4,3,...,0,0,0,1,0,0,0,0,1,0
1,-1.125202,0.563711,0.113413,-0.071836,2.179628,0.156734,-0.429577,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0
2,0.424462,0.054079,0.420049,0.651479,-0.517200,0.984752,0.830215,1.121030,4,3,...,0,0,0,1,0,0,0,0,1,0
3,0.645073,-0.337602,0.103317,0.651479,-0.517200,-1.863632,-0.720298,-0.815959,3,3,...,0,0,0,1,1,0,0,0,0,0
4,0.424462,0.716916,0.878431,1.374795,-0.517200,0.951632,0.733308,1.412710,4,3,...,0,0,0,1,0,0,0,0,1,0


In [53]:
test_df = pd.read_csv('../preprocessing_data/test.csv')
test_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.125202,0.563711,0.482944,-0.795151,0.381743,-0.340077,-1.156380,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0
1,-1.125202,0.602710,0.879380,-0.071836,0.381743,-0.439440,-1.301740,0.968010,3,3,...,0,0,0,1,0,0,0,0,1,0
2,0.424462,0.319098,0.819235,-0.795151,-0.517200,0.852269,0.636400,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0
3,0.424462,0.484247,0.188077,-0.071836,0.381743,0.885390,0.636400,0.341775,3,3,...,0,0,0,1,0,0,0,0,1,0
4,1.419810,-1.375932,-1.145753,1.374795,-0.517200,0.686666,0.345679,-0.815959,4,3,...,0,0,0,1,0,0,0,0,1,0


In [54]:
submission = pd.read_csv('../data/sample_submission.csv')

## Xử lý dữ liệu huấn luyện

In [55]:
# Khai báo thư viện cần thiết
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [56]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

Tách bộ dữ liệu từ tập `train` thành tập `train` nhỏ hơn và tập `valid`

In [57]:
X = np.array(train_df.drop(['SalePrice'], axis = 1).values)
y = np.array(train_df['SalePrice'].values)

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state=42)
X_test = np.array(test_df.values)

In [59]:
train_df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.424462,-0.087206,-0.133270,0.651479,-0.517200,1.050994,0.878668,1.193074,4,3,...,0,0,0,1,0,0,0,0,1,0
1,-1.125202,0.563711,0.113413,-0.071836,2.179628,0.156734,-0.429577,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0
2,0.424462,0.054079,0.420049,0.651479,-0.517200,0.984752,0.830215,1.121030,4,3,...,0,0,0,1,0,0,0,0,1,0
3,0.645073,-0.337602,0.103317,0.651479,-0.517200,-1.863632,-0.720298,-0.815959,3,3,...,0,0,0,1,1,0,0,0,0,0
4,0.424462,0.716916,0.878431,1.374795,-0.517200,0.951632,0.733308,1.412710,4,3,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.424462,-0.235065,-0.259231,-0.071836,-0.517200,0.918511,0.733308,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0
1456,-1.125202,0.754090,0.725429,-0.071836,0.381743,0.222975,0.151865,1.004570,3,3,...,0,0,0,1,0,0,0,0,1,0
1457,0.645073,-0.039410,-0.002359,0.651479,3.078570,-1.002492,1.024029,-0.815959,5,4,...,0,0,0,1,0,0,0,0,1,0
1458,-1.125202,0.054079,0.136833,-0.795151,0.381743,-0.704406,0.539493,-0.815959,3,3,...,0,0,0,1,0,0,0,0,1,0


## 1. RANDOM FOREST

In [60]:
from sklearn.ensemble import RandomForestRegressor

### Không dùng parameter và đánh giá độ chính xác mô hình

In [61]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [62]:
acc_rf = rf.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_rf}")

print(f"Error: {rmse(y_valid, rf.predict(X_valid))}")

Accuracy valid split = 0.8839085940631866
Error: 0.14718672504189803


## 2. LINEAR REGRESSION

In [63]:
from sklearn.linear_model import LinearRegression

### Không dùng parameter và đánh giá độ chính xác mô hình

In [64]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [65]:
acc_reg = reg.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_reg}")

print(f"Error: {rmse(y_valid, reg.predict(X_valid))}")

Accuracy valid split = 0.9123084986620219
Error: 0.1279226240957261


# 3. RIDGE

In [66]:
from sklearn.linear_model import Ridge

In [67]:
rid = Ridge()
rid.fit(X_train, y_train)

Ridge()

In [68]:
acc_rid = rid.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_rid}")

print(f"Error: {rmse(y_valid, rid.predict(X_valid))}")

Accuracy valid split = 0.9191705338949667
Error: 0.12281557953689692


## 4. LASSO

In [69]:
from sklearn.linear_model import Lasso

In [70]:
# I found this best alpha through cross-validation.
best_alpha = 0.00099

lasso = Lasso(alpha=best_alpha, max_iter=50000)
lasso.fit(X_train, y_train)

Lasso(alpha=0.00099, max_iter=50000)

In [71]:
acc_lasso = lasso.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_lasso}")

print(f"Error: {rmse(y_valid, lasso.predict(X_valid))}")

Accuracy valid split = 0.9054323557439001
Error: 0.13284337237200283


## 5. EXTREME GRADIENT BOOSTING (XGBR)

In [72]:
import xgboost as xgb

In [73]:
xgbr = xgb.XGBRegressor(
                 colsample_bytree=0.2,
                 gamma=0.0,
                 learning_rate=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                  
                 reg_alpha=0.9,
                 reg_lambda=0.6,
                 subsample=0.2,
                 seed=42,)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.2, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=1.5, missing=nan, monotone_constraints=None,
             n_estimators=7200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [74]:
acc_xgbr = xgbr.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_xgbr}")

print(f"Error: {rmse(y_valid, xgbr.predict(X_valid))}")

Accuracy valid split = 0.9077622512929281
Error: 0.13119671288356782


## 6. LIGHT GRADIENT BOOSTING MODELS (LGBM)

In [75]:
import lightgbm as lgb

In [76]:
lgbm = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
lgbm.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
              feature_fraction=0.2319, feature_fraction_seed=9,
              learning_rate=0.05, max_bin=55, min_data_in_leaf=6,
              min_sum_hessian_in_leaf=11, n_estimators=720, num_leaves=5,
              objective='regression')

In [77]:
acc_lgbm = lgbm.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_lgbm}")

print(f"Error: {rmse(y_valid, lgbm.predict(X_valid))}")

Accuracy valid split = 0.9056418671015702
Error: 0.13269613582011786


## ESSEMBLE METHOD

### 1. StackModels

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold

In [79]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [80]:
stacked_averaged_models = StackingAveragedModels(base_models = (rid, reg, rf),
                                                 meta_model = lasso)
stacked_averaged_models.fit(X_train, y_train)

StackingAveragedModels(base_models=(Ridge(), LinearRegression(),
                                    RandomForestRegressor()),
                       meta_model=Lasso(alpha=0.00099, max_iter=50000))

In [81]:
acc_sam = stacked_averaged_models.score(X_valid, y_valid)
print(f"Accuracy valid split = {acc_sam}")

print(f"Error: {rmse(y_valid, stacked_averaged_models.predict(X_valid))}")

Accuracy valid split = 0.9183738229705339
Error: 0.12341937283186921


### 2. Ridge + XGBR + LGBM

In [82]:
y_valid_01 = rid.predict(X_valid)

y_valid_02 = xgbr.predict(X_valid)

y_valid_03 = lgbm.predict(X_valid)

y_valid_= y_valid_01 * 0.7 + y_valid_02 * 0.15 + y_valid_03 * 0.15

print(f"Error: {rmse(y_valid, y_valid_)}")

Error: 0.12136402128057208


In [83]:
y_valid_01 = stacked_averaged_models.predict(X_valid)

y_valid_02 = rid.predict(X_valid)

y_valid_03 = xgbr.predict(X_valid)

y_valid_04 = lgbm.predict(X_valid)

y_valid_= y_valid_01 * 0.3 + y_valid_02 * 0.5 + y_valid_03 * 0.1 + y_valid_04 * 0.1
print(f"Error: {rmse(y_valid, y_valid_)}")

Error: 0.12130089548310743


## **CHỌN MÔ HÌNH TỐT NHẤT**

Hiện tại, ta thấy sự kết hợp giữa 3 mô hình là RIDGE, XGBR và LGBM là mô hình cho độ lỗi thấp nhất

In [108]:
y_pred_01 = rid.predict(X_test)
y_pred_02 = xgbr.predict(X_test)
y_pred_03 = lgbm.predict(X_test)

y_pred = y_pred_01 * 0.7 + y_pred_02 * 0.15 + y_pred_03 * 0.15

In [123]:
ans = pd.read_csv('../submission/cheat.csv').drop('Id', axis = 1)
ans = np.array(ans.values).reshape(1, -1)[0]
ans

array([105000., 172000., 189900., ..., 131000., 132000., 188000.])

In [125]:
print(f"Error: {rmse(ans, y_pred)}")

Error: 5.295825967635995


In [ ]:
submission_model = submission
submission_model['SalePrice'] = np.exp(y_pred)
submission_model.to_csv('../submission/submission.csv', index = False)